In [46]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('2018_new.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY:

In [47]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
OP_CARRIER,UA,UA,UA,UA,UA
OP_CARRIER_FL_NUM,2429,2427,2426,2425,2424
ORIGIN,EWR,LAS,SNA,RSW,ORD
DEST,DEN,SFO,DEN,ORD,ALB
CRS_DEP_TIME,1517,1115,1335,1546,630
DEP_TIME,1512,1107,1330,1552,650
DEP_DELAY,-5,-8,-5,6,20
TAXI_OUT,15,11,15,19,13
WHEELS_OFF,1527,1118,1345,1611,703


In [48]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
OP_CARRIER,UA,UA,UA,UA,UA
OP_CARRIER_FL_NUM,2429,2427,2426,2425,2424
ORIGIN,EWR,LAS,SNA,RSW,ORD
DEST,DEN,SFO,DEN,ORD,ALB
CRS_DEP_TIME,1517,1115,1335,1546,630
DEP_TIME,1512,1107,1330,1552,650
DEP_DELAY,-5,-8,-5,6,20
TAXI_OUT,15,11,15,19,13
WHEELS_OFF,1527,1118,1345,1611,703


In [49]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
OP_CARRIER_FL_NUM,7213446,2607.5313346769353,1860.1222647858185,1,7909
CRS_DEP_TIME,7213446,1329.6870178275403,490.9319815381578,1,2359
CRS_ARR_TIME,7213446,1486.341099108526,518.3124279417913,1,2400
Delayed,7213446,0.33953494626562675,0.4735514730395628,0,1


In [68]:
df = df.select('OP_CARRIER', 'DEP_DELAY', 'ARR_DELAY','Delayed')
cols = df.columns
df.printSchema()

root
 |-- OP_CARRIER: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- Delayed: integer (nullable = true)



In [69]:
from csv import writer
from csv import reader
def data_append(row, line_num):
    if line_num==1:
        return str(row.append('Delayed'))
    delay = row[8]
    dataApp=''
    if(delay==''):
        return str(row.append('0'))
    elif float(delay)>0.0:
        return str(row.append('1'))
    else:
        return str(row.append('0'))

def add_column_in_csv(input_file, output_file, transform_row):
    """ Append a column in existing csv using csv.reader / csv.writer classes"""
    # Open the input_file in read mode and output_file in write mode
    with open(input_file, 'r') as read_obj, \
            open(output_file, 'w', newline='') as write_obj:
        # Create a csv.reader object from the input file object
        csv_reader = reader(read_obj)
        # Create a csv.writer object from the output file object
        csv_writer = writer(write_obj)
        # Read each row of the input csv file as list
        for row in csv_reader:
            # Pass the list / row in the transform function to add column text for this row
            transform_row(row, csv_reader.line_num)
            # Write the updated row / list to the output file
            csv_writer.writerow(row)

In [45]:
add_column_in_csv('2018_regression.csv', '2018_new.csv', lambda row, line_num: row.append('Delayed') if line_num==1 else (row.append('0') if row[7]=='' else (row.append('1') if(float(row[7])>0.0) else row.append('0'))))

In [70]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['OP_CARRIER']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'Delayed', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['DEP_DELAY', 'ARR_DELAY']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [71]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- Delayed: integer (nullable = true)



In [82]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,0,0,0,1,1
features,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
OP_CARRIER,UA,UA,UA,UA,UA
DEP_DELAY,-5,-8,-5,6,20
ARR_DELAY,-23,-24,-13,-2,14
Delayed,0,0,0,1,1


In [83]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

In [84]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

Py4JJavaError: An error occurred while calling o1073.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 68.0 failed 1 times, most recent failure: Lost task 7.0 in stage 68.0 (TID 393) (DESKTOP-VQUIMAF executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$4303/1800967155: (struct<OP_CARRIERclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,DEP_DELAY:double,ARR_DELAY:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressi

In [76]:
predictions = lrModel.transform(test)
predictions.select('OP_CARRIER', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

NameError: name 'lrModel' is not defined